In [ ]:
import os
os.chdir('..')

In [ ]:
from src.data_prep.load_data import (
    get_league_data,
    get_team_history,
    get_league_history,
    get_current_season_information,
    get_managers_information_league
)
from src.data_prep.reshape_data import (
    summarise_season_current,
    summarise_season_history
)
from src.data_prep.output_data import (
    filter_rehsaped_season_history
)

In [ ]:
#Variables
season_start_year = 2001
league_id=

In [ ]:
league_data, team_data = get_league_data(league_id=league_id)
manager_information = get_managers_information_league(team_data=team_data)
season_history = get_league_history(team_data=team_data)
final_gw_finished, current_season_year, team_ids = get_current_season_information()

season_current_df = summarise_season_current(
        league_data=league_data,
        team_data=team_data,
        manager_information=manager_information,
        current_season_year=current_season_year,
        team_ids=team_ids
        )

season_history_df = summarise_season_history(
        season_history=season_history,
        season_current_df=season_current_df,
        final_gw_finished=final_gw_finished)

season_history_df_filtered = filter_rehsaped_season_history(
    season_start_year=season_start_year,
    df=season_history_df
) 



In [ ]:
# Get max value
max_points_season_index = season_history_df_filtered.groupby('team_name')['total_points'].idxmax()

# Create a DataFrame with both 'team' and 'ranking' columns for each group
max_points_season_df = season_history_df_filtered.loc[max_points_season_index, ['team_name', 'manager_name','season_name']]
max_points_season_df.rename(columns={'season_name': 'max_points_season_year'}, inplace=True)

# Get max value
min_rank_season_index = season_history_df_filtered.groupby('team_name')['rank'].idxmin()

# Create a DataFrame with both 'team' and 'ranking' columns for each group
min_rank_season_df = season_history_df_filtered.loc[min_rank_season_index, ['team_name','manager_name', 'season_name']]
min_rank_season_df.rename(columns={'season_name': 'min_rank_season_year'}, inplace=True)

In [ ]:
def get_seasons_by_positions(df,position,column_name):
    """"
    """
    df=df[df['league_position']==position].groupby(['team_name','manager_name'])['season_name'].apply(lambda x: ', '.join(map(str, x))).reset_index(name=column_name)

    return df


In [ ]:
# Group by 'team' and aggregate values of 'score' column into a string
seasons_played = season_history_df_filtered.groupby(['team_name','manager_name'])['season_name'].apply(lambda x: ', '.join(map(str, x))).reset_index(name='seasons_played_years')

seasons_first = get_seasons_by_positions(
    df=season_history_df_filtered,
    position=1,
    column_name='seasons_won_years'
    )

seasons_second = get_seasons_by_positions(
    df=season_history_df_filtered,
    position=2,
    column_name='seasons_runner_up_years'
    )

seasons_third = get_seasons_by_positions(
    df=season_history_df_filtered,
    position=3,
    column_name='seasons_third_years'
    )

In [ ]:
seasons_overview = season_history_df_filtered.groupby(['team_id','team_name','manager_name']).agg(
    seasons_won=('league_position', lambda x: (x == 1).sum()),
    seasons_runner_up=('league_position', lambda x: (x == 2).sum()),
    seasons_third=('league_position', lambda x: (x == 3).sum()),
    seasons_played=('season_name', 'nunique'),
    maximum_points=('total_points', 'max'),
    minimum_rank=('rank', 'min')
).reset_index()

In [ ]:
# Join together
seasons_overview=seasons_overview.merge(
    right=max_points_season_df,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=min_rank_season_df,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=seasons_played,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=seasons_first,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=seasons_second,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=seasons_third,
    on=['team_name','manager_name'],
    how='left'
).merge(
    right=pd.DataFrame(manager_information),
    left_on=['team_id'],
    right_on=['entry'],
    how='left'
).sort_values([
    'seasons_won','seasons_runner_up','seasons_third'
],ascending=False)

# Add rank
seasons_overview['rank']=seasons_overview['seasons_won'].rank(method='min',ascending=False).astype('int')


In [ ]:
### Summary overall
# Most wins
# Best rank overall (and season)
# Best score overall (and season)


### Summarise by grouping by team
# Seasons played number
# Total wins
# Total runner up
# Total 3rd
# Best Points
# Best Points Season
# Best Rank
# Best Rank Season
# Nationality
# Favourite team
# Seasons played list 
# Years won


### Summarise by grouping by season
# 1st team name
# 1st total points
# 1st overall rank
# 2nd
# 3rd